<a href="https://colab.research.google.com/github/Mefoolyhi/JokeGenieBot/blob/main/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-telegram-bot requests

In [1]:
!pip install python-telegram-bot==13.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3


In [3]:
!pip install faiss-gpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import csv
import random
import os
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler, CallbackContext
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
from nltk.tokenize import NLTKWordTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Токен вашего бота
TOKEN = '7065562212:AAHiu-AYQ9ggAz9f0jlJoGNu_vM4FQPzDpQ'

# Файл с анекдотами
JOKES_FILE = 'total_data_jokes_512.csv'
# Файл для хранения истории оценок пользователей
HISTORY_FILE = 'user_favorites.csv'

# Словари для хранения понравившихся, не понравившихся и просмотренных анекдотов для каждого пользователя
user_favorites = {}
user_dislikes = {}
user_viewed = {}
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
encoder = SentenceTransformer('bert-base-nli-mean-tokens')

# Функция для загрузки анекдотов из CSV-файла
def load_jokes(filename):
    total_data = pd.read_csv('total_data_jokes_512.csv', encoding = "utf-8")
    jokes = total_data['joke']
    encoded_data = encoder.encode(jokes) # это будет работать минут 10, это нормально
    index.add_with_ids(encoded_data, np.arange(len(jokes)))
    return jokes

# Загрузка анекдотов при старте
jokes = load_jokes(JOKES_FILE)

# Функция для загрузки истории оценок пользователей из CSV-файла
def load_user_favorites(filename):
    if os.path.exists(filename):
        with open(filename, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # Пропустить заголовок
            for row in reader:
                if row:  # Проверка, что строка не пустая
                    try:
                        user_id = int(row[0])
                        likes = row[1].split('|') if row[1] else []
                        dislikes = row[2].split('|') if row[2] else []
                        viewed = row[3].split('|') if row[3] else []
                        user_favorites[user_id] = likes
                        user_dislikes[user_id] = dislikes
                        user_viewed[user_id] = viewed
                    except ValueError:
                        print(f"Ошибка преобразования user_id в целое число: {row[0]}")
    else:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['user_id', 'favorites', 'dislikes', 'viewed'])

# Сохранение истории оценок пользователей в CSV-файл
def save_user_favorites(filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['user_id', 'favorites', 'dislikes', 'viewed'])
        for user_id in user_favorites:
            favorites = '|'.join(user_favorites.get(user_id, []))
            dislikes = '|'.join(user_dislikes.get(user_id, []))
            viewed = '|'.join(user_viewed.get(user_id, []))
            writer.writerow([user_id, favorites, dislikes, viewed])

# Загрузка истории оценок при старте
load_user_favorites(HISTORY_FILE)

def start(update: Update, context: CallbackContext) -> None:
    print("Received start command!")  # Выводим сообщение о получении команды "start"
    user_id = update.message.from_user.id
    if user_id not in user_favorites:
        user_favorites[user_id] = []
    if user_id not in user_dislikes:
        user_dislikes[user_id] = []
    if user_id not in user_viewed:
        user_viewed[user_id] = []

    # Инициализация списка просмотренных анекдотов
    context.user_data['viewed_jokes'] = []

    keyboard = [
        [InlineKeyboardButton("Показать анекдот", callback_data='show_joke')],
        [InlineKeyboardButton("Понравившиеся анекдоты", callback_data='show_favorites')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text('Привет! Нажми кнопку ниже, чтобы получить анекдот или посмотреть понравившиеся анекдоты.', reply_markup=reply_markup)

def show_joke(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    if user_id not in user_favorites:
        user_favorites[user_id] = []
    if user_id not in user_dislikes:
        user_dislikes[user_id] = []
    if user_id not in user_viewed:
        user_viewed[user_id] = []

    # Инициализируем 'viewed_jokes', если он еще не был создан
    if 'viewed_jokes' not in context.user_data:
        context.user_data['viewed_jokes'] = []

    viewed_jokes = context.user_data['viewed_jokes']

    last_liked = user_favorites[user_id][-5:]


    if last_liked:
        query_vector = encoder.encode(last_liked)
        top_k = index.search(query_vector, 10)
        top_jokes = []
        for _id in top_k[1][0]:
            joke = jokes[_id]
            if joke not in viewed_jokes:
                top_jokes.append(joke)

        unliked = user_dislikes[user_id]
        if unliked and top_k:
            tokenizer = NLTKWordTokenizer()
            unliked_tok = [tokenizer.tokenize(x.lower()) for x in unliked]
            top_jokes_tok = [tokenizer.tokenize(x.lower()) for x in top_jokes]

            lemmatizer = WordNetLemmatizer()

            russian_stopwords = stopwords.words("russian")
            unliked_tokens = []
            for row in unliked_tok:
                row_tokens = []
                for token in row:
                    if token not in russian_stopwords and token not in punctuation:
                        row_tokens.append(lemmatizer.lemmatize(token))
                unliked_tokens.append(' '.join(row_tokens))

            top_tokens = []
            for row in top_jokes_tok:
                row_tokens = []
                for token in row:
                    if token not in russian_stopwords and token not in punctuation:
                        row_tokens.append(lemmatizer.lemmatize(token))
                top_tokens.append(row_tokens)


            tf_idf = TfidfVectorizer()
            unliked_array = tf_idf.fit_transform(unliked_tokens)
            threshold = 0.2 # тут его возможно надо будет поднять
            important_words = [word for word, score in zip(tf_idf.get_feature_names_out(), unliked_array.toarray()[0]) if score >= threshold]
            important_set = set(important_words)
            counts = []
            for joke in top_tokens:
                local_count = 0
                for token in joke:
                    if token in important_words:
                        local_count += 1
                counts.append(local_count)

            final_joke = top_jokes[np.argmin(counts)]
        else:
            final_joke = random.choice(top_jokes)
    else:
        available_jokes = [j for j in jokes if j not in viewed_jokes]

        if not available_jokes:
            query.message.reply_text("Все анекдоты уже показаны!")
            return

        final_joke = random.choice(available_jokes)


    context.user_data['current_joke'] = final_joke
    context.user_data['viewed_jokes'].append(final_joke)
    user_viewed[user_id].append(final_joke)

    # Сохранение истории оценок
    save_user_favorites(HISTORY_FILE)

    keyboard = [
        [InlineKeyboardButton("👍 Понравился", callback_data='like'),
         InlineKeyboardButton("👎 Не понравился", callback_data='dislike')],
        [InlineKeyboardButton("Еще", callback_data='show_joke')],
        [InlineKeyboardButton("Понравившиеся анекдоты", callback_data='show_favorites')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    # Отправляем анекдот
    query.message.reply_text(text=final_joke, reply_markup=reply_markup)

def show_favorites(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    if user_id not in user_favorites or not user_favorites[user_id]:
        query.edit_message_text(text="У вас нет понравившихся анекдотов.")
        # тут не хватает кнопок
        return

    favorites = user_favorites[user_id]
    favorites_text = "\n\n".join(favorites)

    # Показываем список понравившихся анекдотов
    query.edit_message_text(text=f"Ваши понравившиеся анекдоты:\n\n{favorites_text}")

    # Добавляем кнопку "Показать анекдот"
    keyboard = [
        [InlineKeyboardButton("Показать анекдот", callback_data='show_joke')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.message.reply_text('Нажми кнопку ниже, чтобы получить анекдот.', reply_markup=reply_markup)

def handle_feedback(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    user_id = query.from_user.id

    if user_id not in user_favorites:
        user_favorites[user_id] = []
    if user_id not in user_dislikes:
        user_dislikes[user_id] = []
    if user_id not in user_viewed:
        user_viewed[user_id] = []

    feedback = query.data
    current_joke = context.user_data.get('current_joke', None)

    if current_joke:
        if feedback == 'like':
            user_favorites[user_id].append(current_joke)
        elif feedback == 'dislike':
            user_dislikes[user_id].append(current_joke)

        # Сохранение истории оценок
        save_user_favorites(HISTORY_FILE)

    # Показываем благодарность за отзыв
    query.message.reply_text("Спасибо за отзыв!")

    # Добавляем кнопку "Показать анекдот"
    keyboard = [
        [InlineKeyboardButton("Показать анекдот", callback_data='show_joke')],
        [InlineKeyboardButton("Понравившиеся анекдоты", callback_data='show_favorites')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    query.message.reply_text('Нажми кнопку ниже, чтобы получить анекдот.', reply_markup=reply_markup)

def main() -> None:
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CallbackQueryHandler(show_joke, pattern='show_joke'))
    dispatcher.add_handler(CallbackQueryHandler(handle_feedback, pattern='like|dislike'))
    dispatcher.add_handler(CallbackQueryHandler(show_favorites, pattern='show_favorites'))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
